In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from random import randint
from numpy import array, argmax, asarray, zeros
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
nlp=spacy.load("en_core_web_md")
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding,GRU, Bidirectional
from keras.models import Sequential
from keras.layers import Flatten, Masking
from keras.layers.recurrent import SimpleRNN, LSTM
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from string import digits
pd.options.mode.chained_assignment = None
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords, wordnet
from nltk import punkt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import Markdown, display
import gensim.utils
from xgboost import XGBClassifier
from gensim.parsing.preprocessing import remove_stopwords
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 24
np.random.seed(RANDOM_SEED)
#torch.manual_seed(RANDOM_SEED)

Using TensorFlow backend.


### Enter your product query below

In [3]:
#turn query product into a dataframe (list of strings)
brand = ['Fila']
product_full_name = ['Trainer sneakers']
description = ['leather white trainers with thick shoelaces']
brand_category = ['Sneakers, Shoes']
details = ['1dsaf']

input_df = pd.DataFrame([brand,
                         product_full_name,
                         description,
                         brand_category,
                         details]).T

In [ ]:
input_df.columns = ['brand','product_full_name','description','brand_category','details']

In [4]:
#read in data
full_products_df = pd.read_csv('full_data.csv')
tagged_df = pd.read_csv('tagged_product_attributes.csv')

In [5]:
#merge on product data & lower-case description
tagged_products_df = tagged_df.merge(full_products_df, left_on='product_id', right_on='product_id')
tagged_products_df['description'] = tagged_products_df['description'].str.lower()
input_df['description'] = input_df['description'].str.lower()

In [6]:
# fill empty description with product name
def fill_nas(row):
  if type(row['description']) == str:
    return row['description']
  if np.isnan(row['description']):
    return row['product_full_name']
  return row['description']

In [7]:
#fill nas
tagged_products_df['description'] = tagged_products_df.apply(fill_nas,axis=1)
tagged_products_df.brand_category = tagged_products_df.brand_category.fillna('unknown')

input_df['description'] = input_df.apply(fill_nas,axis=1)
input_df.brand_category = input_df.brand_category.fillna('unknown')

In [8]:
#regex cleaning for brand category
tagged_products_df.brand_category = tagged_products_df.brand_category.\
                          apply(lambda x: ','.join(pd.Series\
                          (re.findall(r'[/:]?(\w+[\s:]\w+[\w+:\w+]*[\s]\w*|\w+[\s:]\w+[\w+:\w+]*|\w+)[/:]?', str(x).lower()))\
                          .drop_duplicates().tolist()))

input_df.brand_category = input_df.brand_category.\
                          apply(lambda x: ','.join(pd.Series\
                          (re.findall(r'[/:]?(\w+[\s:]\w+[\w+:\w+]*[\s]\w*|\w+[\s:]\w+[\w+:\w+]*|\w+)[/:]?', str(x).lower()))\
                          .drop_duplicates().tolist()))

In [9]:
# make a set of all the categories
def add_cat(x):
  cats = re.findall(r'[,]?(\w+[\s]\w+[\s]\w+|\w+[\s]?\w+|\w+)[,]?',x)
  for cat in cats:
    category.add(cat)
  return

In [10]:
#remove duplicates
def remove_dupe(alist):
  blist = []
  for cat in alist:
    blist.append(cat.strip())
  return ','.join(pd.Series(blist).drop_duplicates().tolist())

In [11]:
#remove duplicates in a single line of brand category
remove_digits = str.maketrans('', '', digits)
tagged_products_df.brand_category = tagged_products_df.brand_category\
                  .apply(lambda x: x.replace(':', ','))\
                  .apply(lambda x: x.translate(remove_digits))\
                  .apply(lambda x: remove_dupe(x.split(',')))
category = set()
tagged_products_df.brand_category.apply(add_cat)

input_df.brand_category = input_df.brand_category\
                  .apply(lambda x: x.replace(':', ','))\
                  .apply(lambda x: x.translate(remove_digits))\
                  .apply(lambda x: remove_dupe(x.split(',')))

In [12]:
#merge duplicate categories into ones
to_replace = [[r'(\bt,shirts\b)','tees'],
[r'(\baccessories\b|\baccessory\b|\ball_accessories\b)','accessories'],
[r'(\bbelt bags\b|\bbeltbags\b)','beltbags'],
[r'(\bcoatsjacketswa\b)','coats_and_jackets'],
[r'(\bcross body\b|\bcrossbody\b)','crossbody'],
[r'(\bflat shoes\b|\bflat\b)','flats'],
[r'(\bhandbagsshoes\b)','handbags,shoes'],
[r'(\bjacketsvests\b)','jackets,vests'],
[r'(\bjackets\b|\bjacket\b)','jackets'],
[r'(\bdressesandjumpsuits\b)','dresses,jumpsuits'],
[r'(\bbolerosjacketsvests\b)','boleros,jackets,vests'],
[r'(\bapparelaccessories\b)','apparel,accessories'],
[r'(\blow top\b|\blowtop\b)','lowtop'],
[r'(\bmini bags\b)','minibags'],
[r'(\bpantsshortsjumpsuits\b)','pants,shorts,jumpsuits'],
[r'(\bshirts_tops\b)','shirts,tops'],
[r'(\bshorts\b)','short'],
[r'(\bshoulder bags\b|\bshoulder_bags\b)','shoulderbags'],
[r'(\bsweatshirts_sweatpants\b)','sweatshirts,sweatpants'],
[r'(\btshirts_tanktops\b)','tees,tanktops'],
[r'(\btshirtstanks\b)','tees,tanktops'],
[r'(\bwomensapparel\b)','women'],
[r'(\bclutch bags\b)','clutches'],
[r'(\bjumpsuit\b)','jumpsuits'],
[r'(\bmidi dresses\b)','midi'],
[r'(\bromper\b)','rompers'],
[r'(\bskirt\b)','skirts'],
[r'(\bsweater\b)','sweaters'],
[r'(\btote bags\b)','totes'],
[r'(\btop\b)','tops']]

for replace in to_replace:
    tagged_products_df.brand_category = tagged_products_df.brand_category.apply(lambda x: re.sub(replace[0],replace[1], x.lower()))
for replace in to_replace:
    input_df.brand_category = input_df.brand_category.apply(lambda x: re.sub(replace[0],replace[1], x.lower()))

In [13]:
#strip puncs for brands
import string
tagged_products_df.brand = tagged_products_df.brand.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

input_df.brand = input_df.brand.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [14]:
#merge duplicate brands into ones
brand_replace = [[r'®',''],
[r'(\bvictoria victoria beckham\b)','victoria beckham']]

for replace in brand_replace:
    tagged_products_df.brand = tagged_products_df.brand.apply(lambda x: re.sub(replace[0],replace[1], x))
    
for replace in brand_replace:
    input_df.brand = input_df.brand.apply(lambda x: re.sub(replace[0],replace[1], x))

In [15]:
#drop duplicates
tagged_products_df = tagged_products_df[['attribute_name','attribute_value','brand','product_full_name','description','brand_category']].drop_duplicates()

input_df = input_df[['brand','product_full_name','description','brand_category', 'details']].drop_duplicates()

# Clean specific DF

In [16]:
# make specific datatframes
style_df = tagged_products_df[tagged_products_df['attribute_name']=='style']
attributes_df = tagged_products_df[tagged_products_df['attribute_name']=='occasion']
input_style_df =input_df.drop('details', axis=1)
input_occasion_df =input_df.drop('details', axis=1)

In [17]:
attributes_df = attributes_df[['attribute_value', 'brand','product_full_name','description','brand_category']]
style_df = style_df[['attribute_value', 'brand','product_full_name','description','brand_category']]
input_style_df =input_style_df[['brand','product_full_name','description','brand_category']]
input_occasion_df =input_occasion_df[['brand','product_full_name','description','brand_category']]

In [18]:
#make attribute values lowercase
attributes_df.attribute_value = attributes_df.attribute_value.str.lower()
style_df.attribute_value = style_df.attribute_value.str.lower()

In [19]:
#merge similar attributes into ones
attributes_df.attribute_value = attributes_df.attribute_value.str.replace('day to night','daytonight').str.replace('night out','nightout')
style_df.attribute_value = style_df.attribute_value.str.replace('businesscasual','business casual')

In [20]:
#1 hot encoding for responses
for style in style_df.attribute_value.unique():
    style_df[f'{style}_target'] = style_df['attribute_value'] == style
for attribute in attributes_df.attribute_value.unique():
    attributes_df[f'{attribute}_target'] = attributes_df['attribute_value'] == attribute

In [21]:
style_df = style_df.drop_duplicates()
attributes_df = attributes_df.drop_duplicates()

In [22]:
#processing same items with multiple rows
styleindex = style_df.set_index(['brand','product_full_name','description','brand_category']).index

for ind in styleindex:
  brand, name, description, brand_category = ind
  values = style_df.set_index(['brand','product_full_name','description','brand_category']).loc[ind]['attribute_value']
  for value in values:
    m = (style_df['brand']==brand)\
        &(style_df['product_full_name']==name)\
        &(style_df['description']==description)\
        &(style_df['brand_category']==brand_category)
    style_df.loc[m, f'{value}_target'] = True

style_df = style_df.drop('attribute_value',axis=1).drop_duplicates()    

In [23]:
attributeindex = attributes_df.set_index(['brand','product_full_name','description','brand_category']).index
for ind in attributeindex:
  brand, name, description, brand_category = ind
  values = attributes_df.set_index(['brand','product_full_name','description','brand_category']).loc[ind]['attribute_value']
  for value in values:
    m = (attributes_df['brand']==brand)\
        &(attributes_df['product_full_name']==name)\
        &(attributes_df['description']==description)\
        &(attributes_df['brand_category']==brand_category)
    attributes_df.loc[m, f'{value}_target'] = True
    
attributes_df = attributes_df.drop('attribute_value',axis=1).drop_duplicates()

In [24]:
attributes = [col.replace('_target','') for col in attributes_df.columns if 'target' in col]
styles = [col.replace('_target','') for col in style_df.columns if 'target' in col]

In [25]:
for attribute in attributes:
  attributes_df[f'{attribute}_target'] = attributes_df[f'{attribute}_target'].astype(float)
for style in styles:
  style_df[f'{style}_target'] = style_df[f'{style}_target'].astype(float)

In [26]:
def one_hot_category(categories, cat):
  for c in categories.split(','):
    if c == cat:
      return 1
  return 0

for cat in category:
  attributes_df[cat] = attributes_df['brand_category'].apply(lambda x: one_hot_category(x, cat))

for cat in category:
  style_df[cat] = style_df['brand_category'].apply(lambda x: one_hot_category(x, cat))

for cat in category:
  input_style_df[cat] = input_style_df['brand_category'].apply(lambda x: one_hot_category(x, cat))

for cat in category:
  input_occasion_df[cat] = input_occasion_df['brand_category'].apply(lambda x: one_hot_category(x, cat))

In [27]:
#one hot encoding for brands
attribute_brand_le = LabelEncoder()
attributes_df['brand_le'] = attribute_brand_le.fit_transform(attributes_df['brand'].values)

style_brand_le = LabelEncoder()
style_df['brand_le'] = style_brand_le.fit_transform(style_df['brand'].values)

style_mapping = style_df.set_index(['brand','brand_le']).index.unique()
style_mapping = {brand[0]:brand[1] for brand in style_mapping}

occasion_mapping = attributes_df.set_index(['brand','brand_le']).index.unique()
occasion_mapping = {brand[0]:brand[1] for brand in occasion_mapping}


input_style_df['brand_le'] = input_style_df['brand'].map(style_mapping)
input_occasion_df['brand_le'] = input_occasion_df['brand'].map(occasion_mapping)

In [28]:
#strip puncs for product full names
attributes_df.product_full_name = attributes_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
style_df.product_full_name = style_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
input_style_df.product_full_name = input_style_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
input_occasion_df.product_full_name = input_occasion_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [29]:
# double check for duplicates
attributes_df.drop_duplicates(inplace=True)
style_df.drop_duplicates(inplace=True)

In [30]:
# lemmatizer function
lem = WordNetLemmatizer()
def lem_sentences(sentence):
    tokens = gensim.utils.tokenize(sentence)
    lemmed_tokens = [lem.lemmatize(token) for token in tokens]
    return ' '.join(lemmed_tokens)

In [31]:
#strip puncs for descrps
import string
attributes_df.description = attributes_df.description.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
style_df.description = style_df.description.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

attributes_df.product_full_name = attributes_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
style_df.product_full_name = style_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

input_occasion_df.description = input_occasion_df.description.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
input_style_df.product_full_name = input_style_df.product_full_name.apply(str.lower).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [32]:
#lemmatize descriptions
attributes_df.description = attributes_df.description.apply(lem_sentences)
style_df.description = style_df.description.apply(lem_sentences)

attributes_df.product_full_name = attributes_df.product_full_name.apply(lem_sentences)
style_df.product_full_name = style_df.product_full_name.apply(lem_sentences)

input_occasion_df.description = input_occasion_df.description.apply(lem_sentences)
input_style_df.description = input_style_df.description.apply(lem_sentences)

input_occasion_df.product_full_name = input_occasion_df.product_full_name.apply(lem_sentences)
input_style_df.product_full_name = input_style_df.product_full_name.apply(lem_sentences)

In [33]:
#remove stopwords
attributes_df.description = attributes_df.description.apply(remove_stopwords)
style_df.description = style_df.description.apply(remove_stopwords)

attributes_df.product_full_name = attributes_df.product_full_name.apply(remove_stopwords)
style_df.product_full_name = style_df.product_full_name.apply(remove_stopwords)

input_occasion_df.description = input_occasion_df.description.apply(remove_stopwords)
input_style_df.description = input_style_df.description.apply(remove_stopwords)

input_occasion_df.product_full_name = input_occasion_df.product_full_name.apply(remove_stopwords)
input_style_df.product_full_name = input_style_df.product_full_name.apply(remove_stopwords)

In [34]:
input_occasion_df.fillna(0,inplace=True)
input_style_df.fillna(0,inplace=True)

In [35]:
nlp = spacy.load('en_core_web_md')

#make tfidf function cr to Prof Yu Chen
def tfidf(series):
    vectorizer = TfidfVectorizer(min_df=2,
                                 max_df=0.3)
    X = vectorizer.fit_transform(series)

    tf_idf_lookup_table = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())


    DOCUMENT_SUM_COLUMN = "DOCUMENT_TF_IDF_SUM"

    tf_idf_lookup_table[DOCUMENT_SUM_COLUMN] = tf_idf_lookup_table.sum(axis=1)
    available_tf_idf_scores = tf_idf_lookup_table.columns 
    available_tf_idf_scores = list(map( lambda x: x.lower(), available_tf_idf_scores)) 
    return tf_idf_lookup_table, available_tf_idf_scores

In [36]:
#make embed function cr to Prof Yu Chen
def embed(textdf, available_tf_idf_scores, tf_idf_lookup_table):
    text_vectors = []

    for idx, text in enumerate(textdf): 
        tokens = nlp(text)

        total_tf_idf_score_per_document = 0


        running_total_word_embedding = np.zeros(300) 
        for token in tokens:

       
            if token.has_vector and token.text.lower() in available_tf_idf_scores:

                tf_idf_score = tf_idf_lookup_table.loc[idx, token.text.lower()]
            
                running_total_word_embedding += tf_idf_score * token.vector

                total_tf_idf_score_per_document += tf_idf_score

        document_embedding = running_total_word_embedding / (total_tf_idf_score_per_document + 1e-6)
        text_vectors.append(document_embedding)
    return text_vectors

In [37]:
#tfidf for attribute 
d_tf_idf_lookup_table, d_available_tf_idf_scores = tfidf(attributes_df.description)
d_text_vector = embed(attributes_df.description, d_available_tf_idf_scores, d_tf_idf_lookup_table)
p_tf_idf_lookup_table, p_available_tf_idf_scores = tfidf(attributes_df.product_full_name)
p_text_vector = embed(attributes_df.product_full_name, p_available_tf_idf_scores, p_tf_idf_lookup_table)

In [38]:
#tfidf for style 
d_style_lu_tb, d_style_scores = tfidf(style_df.description)
d_style_vector = embed(style_df.description, d_style_scores, d_style_lu_tb)
p_style_lu_tb, p_style_scores = tfidf(style_df.product_full_name)
p_style_vector = embed(style_df.product_full_name, p_style_scores, p_style_lu_tb)

In [39]:
#combined vec for attribute
combined_vectors = []
for idx in range(len(p_text_vector)):
    combined_vector = 0.2*p_text_vector[idx] + 0.8*d_text_vector[idx]
    combined_vectors.append(combined_vector)

In [40]:
#combined vec for style
style_combined_vectors = []
for idx in range(len(p_style_vector)):
    style_combined_vector = 0.4*p_style_vector[idx] + 0.6*d_style_vector[idx]
    style_combined_vectors.append(style_combined_vector)

In [41]:
#tfidf for input occasion
d_tf_idf_lookup_table_input, d_available_tf_idf_scores_input = tfidf(input_occasion_df.description)
d_input_occasion_vector = embed(input_occasion_df.description, d_available_tf_idf_scores_input, d_tf_idf_lookup_table_input)
p_tf_idf_lookup_table_input, p_available_tf_idf_scores_input = tfidf(input_occasion_df.product_full_name)
p_input_occasion_vector = embed(input_occasion_df.product_full_name, p_available_tf_idf_scores_input, p_tf_idf_lookup_table_input)

input_occasion_combined_vectors = []
for idx in range(len(p_input_occasion_vector)):
    input_occasion_combined_vector = 0.2*p_input_occasion_vector[idx] + 0.8*d_input_occasion_vector[idx]
    input_occasion_combined_vectors.append(input_occasion_combined_vector)

In [42]:
#tfidf for input style
d_tf_idf_lookup_table_input_style, d_available_tf_idf_scores_input_style = tfidf(input_style_df.description)
d_input_style_vector = embed(input_style_df.description, d_available_tf_idf_scores_input_style, d_tf_idf_lookup_table_input_style)
p_tf_idf_lookup_table_input_style, p_available_tf_idf_scores_input_style = tfidf(input_style_df.product_full_name)
p_input_style_vector = embed(input_style_df.product_full_name, p_available_tf_idf_scores_input_style, p_tf_idf_lookup_table_input_style)

input_style_combined_vectors = []
for idx in range(len(p_input_style_vector)):
    input_style_combined_vector = 0.2*p_input_style_vector[idx] + 0.8*d_input_style_vector[idx]
    input_style_combined_vectors.append(input_style_combined_vector)

In [43]:
#reset index so that we can join word embeds later
attributes_df = attributes_df.iloc[:,1:].reset_index().drop('index',axis=1)
style_df = style_df.reset_index().drop('index',axis=1)
input_occasion_df = input_occasion_df.reset_index().drop('index',axis=1)
input_style_df = input_style_df.reset_index().drop('index',axis=1)

In [44]:
#setup models
log_model = LogisticRegression()
svc_model = LinearSVC()
rf_model = RandomForestClassifier(n_estimators=500,min_samples_split=5,min_samples_leaf=3)
knn_model = KNeighborsClassifier(n_neighbors=3)
gb_model = XGBClassifier(n_estimators=150, max_depth=1)

In [45]:
#defining models and targets
models = [log_model, svc_model, rf_model, knn_model, gb_model]
occassions = pd.Series([col if '_target' in col else np.nan for col in attributes_df.columns]).dropna().tolist()
styles = pd.Series([col if '_target' in col else np.nan for col in style_df.columns]).dropna().tolist()

In [46]:
#make predict function
def predict(models, responses, DF, vector, upsample=False, multiclass=False, testsize=0.1):
    from sklearn.metrics import accuracy_score,recall_score,f1_score
    from imblearn.over_sampling import RandomOverSampler
    best_model_dict, prediction_tags = {}, {}
    for response in responses:
        best_score = 0
        best_model,best_predictions = None, None
        for model in models:
            df = DF.copy()
            df = df.loc[:,responses[0]:]
            for i in df.columns:
                if 'target' not in i:
                    slicing_index = i
                    break
            X = pd.concat([df.loc[:,slicing_index:],pd.DataFrame(vector).reindex(df.loc[:,slicing_index:].index)], axis=1)
            y = df[response]
            dataframe = pd.concat([X,y], axis=1).fillna(0)
            y = dataframe[response]
            X = dataframe.iloc[:,:-1]
            X_train, X_test,y_train, y_test = train_test_split(X.values,y.values, random_state=66, test_size=testsize, shuffle=True,stratify=y)
            if upsample:
                ros = RandomOverSampler(random_state=24)
                X_train, y_train = ros.fit_resample(X_train, y_train)
            if multiclass:
                model = OneVsRestClassifier(model)
            model.fit(X_train, y_train)
            prediction = model.predict(X_test)
            accuracy = accuracy_score(y_test, prediction)
            if not multiclass:
                recall = recall_score(y_test, prediction)
                score = f1_score(y_test, prediction)
            else:
                score = accuracy
            if score >= best_score:
                best_score = score
                best_model = model
                best_predictions = prediction
        print(f'The best combined score for {response} is {best_score} with {type(best_model).__name__}')
        best_model_dict[response] = best_model
        prediction_tags[response] = best_predictions
    return best_model_dict, prediction_tags

In [47]:
#training models for occasions
best_occassion_models, best_occassion_predictions = predict(models, occassions, attributes_df, combined_vectors, upsample=True)

The best combined score for work_target is 0.628158844765343 with KNeighborsClassifier
The best combined score for weekend_target is 0.8609271523178809 with LinearSVC
The best combined score for vacation_target is 0.5357142857142858 with LogisticRegression
The best combined score for daytonight_target is 0.8519736842105263 with RandomForestClassifier
The best combined score for nightout_target is 0.5321100917431193 with KNeighborsClassifier
The best combined score for workout_target is 0.5454545454545454 with KNeighborsClassifier
The best combined score for coldweather_target is 0.4918032786885246 with KNeighborsClassifier


In [48]:
#training models for styles
best_style_models, best_style_predictions = predict(models, styles, style_df, style_combined_vectors, upsample=True)

The best combined score for classic_target is 0.5901639344262295 with KNeighborsClassifier
The best combined score for business casual_target is 0.6343612334801763 with XGBClassifier
The best combined score for androgynous_target is 0.5581395348837209 with XGBClassifier
The best combined score for casual_target is 0.8429423459244533 with RandomForestClassifier
The best combined score for modern_target is 0.7075208913649025 with RandomForestClassifier
The best combined score for edgy_target is 0.49746192893401014 with XGBClassifier
The best combined score for romantic_target is 0.5074626865671641 with KNeighborsClassifier
The best combined score for athleisure_target is 0.7272727272727274 with KNeighborsClassifier
The best combined score for boho_target is 0.4512195121951219 with LogisticRegression
The best combined score for glam_target is 0.4895104895104895 with LogisticRegression
The best combined score for retro_target is 0.23076923076923075 with RandomForestClassifier


In [61]:
def integer_encode_documents(docs,tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [62]:
def prep_lstm_df(tag):
        tag_data = pd.read_csv('tagged_product_attributes.csv')
        full_data = pd.read_csv('full_data.csv')
        tag_data['attribute_value']=tag_data['attribute_value'].str.lower()
        tags_df=tag_data[tag_data['attribute_name'].str.lower()==tag]
        tags_1=tags_df.drop_duplicates(subset=['product_id','attribute_value'])
        if tag == 'style':
            tags_1.attribute_value = tags_1.attribute_value.str.replace('businesscasual','business casual')
        if tag == 'occasion':
            tags_1.attribute_value = tags_1.attribute_value.str.replace('day to night','daytonight').str.replace('night out','nightout')
        tags_2=pd.concat([tags_1,pd.get_dummies(tags_1['attribute_value'])],axis=1)
        tags=list(tags_2['attribute_value'].unique())
        full=full_data[['product_id','brand','brand_category','product_full_name','description','details']]\
        .drop_duplicates(subset=['product_id'])
        full=full.fillna('')
        full['input']=full[['brand','brand_category','product_full_name','description','details']].agg(' '.join, axis=1).str.lower()
        full['input'] = full['input'].replace(r"[^a-z0-9\s]+", "")
        out_tag=tags_2.merge(full,how='left',on='product_id')
        out_tag=out_tag.groupby(['product_id','input'])[tags].sum().reset_index()
        out_tag['input']=out_tag['input'].apply(lambda x: lem_sentences(x))
        if tag == 'style':
            out_tag['business casual'] = out_tag['business casual'].replace(2,1)
        if tag == 'occasion':
            out_tag['daytonight'] = out_tag['daytonight'].replace(2,1)
            out_tag['nightout'] = out_tag['nightout'].replace(2,1)
        return out_tag, tags

In [63]:
styles_df, styles = prep_lstm_df('style')

In [64]:
occasion_df, occasions = prep_lstm_df('occasion')

In [65]:
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt', encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float64')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [66]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def make_binary_classification_rnn_model(embedding_matrix, VOCAB_SIZE):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix],trainable=False))
    model.add(Masking(mask_value=0.0)) 
    model.add(Bidirectional(LSTM(units=64)))
    model.add(Dense(16))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy',f1_metric])
    return model

In [67]:
def value_model(i, df):
    inputdata=df['input'].tolist()
    label=df["{}".format(i)].tolist()
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    labels = to_categorical(encoder.fit_transform(label))
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    model = make_binary_classification_rnn_model(embedding_matrix, VOCAB_SIZE)
    model.fit(padded_docs, labels, validation_split = 0.1, epochs=5, verbose=1)
    return model

In [68]:
lstm_style_models = {}
for style in sorted(styles):
    print(f'Predicting for {style}:')
    lstm_style_models[style] = value_model(style, styles_df)

Predicting for androgynous:
Train on 3524 samples, validate on 392 samples
Epoch 1/5
3524/3524 [==============================] - 28s 8ms/step - loss: 0.4333 - accuracy: 0.8121 - f1_metric: 0.8117 - val_loss: 0.2409 - val_accuracy: 0.9362 - val_f1_metric: 0.9255
Epoch 2/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.3691 - accuracy: 0.8351 - f1_metric: 0.8345 - val_loss: 0.2254 - val_accuracy: 0.9362 - val_f1_metric: 0.9255
Epoch 3/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.3377 - accuracy: 0.8482 - f1_metric: 0.8474 - val_loss: 0.2022 - val_accuracy: 0.9362 - val_f1_metric: 0.9255
Epoch 4/5
3524/3524 [==============================] - 26s 7ms/step - loss: 0.3148 - accuracy: 0.8553 - f1_metric: 0.8564 - val_loss: 0.2056 - val_accuracy: 0.9388 - val_f1_metric: 0.9279
Epoch 5/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.2905 - accuracy: 0.8703 - f1_metric: 0.8694 - val_loss: 0.2245 - val_accuracy: 0.9235 - val_f1_

3524/3524 [==============================] - 27s 8ms/step - loss: 0.5697 - accuracy: 0.6918 - f1_metric: 0.6943 - val_loss: 0.6127 - val_accuracy: 0.6811 - val_f1_metric: 0.6707
Epoch 2/5
3524/3524 [==============================] - 26s 7ms/step - loss: 0.4923 - accuracy: 0.7605 - f1_metric: 0.7604 - val_loss: 0.6379 - val_accuracy: 0.6760 - val_f1_metric: 0.6659
Epoch 3/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.4637 - accuracy: 0.7806 - f1_metric: 0.7784 - val_loss: 0.6501 - val_accuracy: 0.6582 - val_f1_metric: 0.6490
Epoch 4/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.4412 - accuracy: 0.7980 - f1_metric: 0.7976 - val_loss: 0.7943 - val_accuracy: 0.6327 - val_f1_metric: 0.6034
Epoch 5/5
3524/3524 [==============================] - 27s 8ms/step - loss: 0.4073 - accuracy: 0.8119 - f1_metric: 0.8133 - val_loss: 0.7160 - val_accuracy: 0.6709 - val_f1_metric: 0.6611
Predicting for retro:
Train on 3524 samples, validate on 392 samples
E

In [69]:
lstm_occasion_models = {}
for occasion in sorted(occasions):
    print(f'{occasion} prediction performance:')
    lstm_occasion_models[occasion] = value_model(occasion, occasion_df)

coldweather prediction performance:
Train on 3522 samples, validate on 392 samples
Epoch 1/5
3522/3522 [==============================] - 27s 8ms/step - loss: 0.2161 - accuracy: 0.9250 - f1_metric: 0.9257 - val_loss: 0.0926 - val_accuracy: 0.9694 - val_f1_metric: 0.9712
Epoch 2/5
3522/3522 [==============================] - 26s 7ms/step - loss: 0.1454 - accuracy: 0.9455 - f1_metric: 0.9459 - val_loss: 0.0906 - val_accuracy: 0.9745 - val_f1_metric: 0.9760
Epoch 3/5
3522/3522 [==============================] - 26s 8ms/step - loss: 0.1265 - accuracy: 0.9503 - f1_metric: 0.9507 - val_loss: 0.1433 - val_accuracy: 0.9388 - val_f1_metric: 0.9423
Epoch 4/5
3522/3522 [==============================] - 26s 7ms/step - loss: 0.1124 - accuracy: 0.9549 - f1_metric: 0.9510 - val_loss: 0.1674 - val_accuracy: 0.9311 - val_f1_metric: 0.9351
Epoch 5/5
3522/3522 [==============================] - 26s 7ms/step - loss: 0.1079 - accuracy: 0.9549 - f1_metric: 0.9552 - val_loss: 0.0805 - val_accuracy: 0.9719 -

In [70]:
# predicting output using traditional ML methods
def ml_predict_output(dataframe, tag, vector):
    df = dataframe.copy()
    for col in df.columns:
        df = dataframe.copy()
        if col in category:
            slicingid = col
            break
    X = pd.concat([df.loc[:,slicingid:],pd.DataFrame(vector).reindex(df.loc[:,slicingid:].index)], axis=1).fillna(0)
    if tag == 'casual':
        return best_style_models[tag+'_target'].predict(X.values)
    return best_occassion_models[tag+'_target'].predict(X.values)

In [71]:
# predicting output using DL methods
def predict_output(input_df, tag, occ=False):
    tf.random.set_seed(RANDOM_SEED)
    input_df = input_df.fillna('')
    input_df['input'] = input_df[['brand','brand_category','product_full_name','description','details']]\
                        .agg(' '.join, axis=1).str.lower()
    input_df['input'] = input_df['input'].replace(r"[^a-z0-9\s]+", "")
    inputdata=input_df['input'].tolist()
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    if occ:
        predictions = [pred[1]>0.5 for pred in lstm_occasion_models[tag].predict(padded_docs)]
    else:
        predictions = [pred[1]>0.5 for pred in lstm_style_models[tag].predict(padded_docs)]
    return predictions

In [72]:
def output_df(input_df, input_occasion_df, input_style_df):
    styles_pred = {}
    occasions_pred = {}
    for style in styles:
        #can add more better performing categories to below list
        #if ML models perform better
        if style in ['casual','modern']: #use tradtional ML for better performance
            styles_pred[style] = ml_predict_output(input_style_df, style, input_style_combined_vectors)
        else:
            styles_pred[style] = predict_output(input_df, style)
    for occasion in occasions:
        if occasion in ['daytonight', 'weekend']: #use tradtional ML for better performance
            occasions_pred[occasion] = ml_predict_output(input_occasion_df, occasion, input_occasion_combined_vectors)
        else:
            occasions_pred[occasion] = predict_output(input_df, occasion, occ=True)
    return styles_pred, occasions_pred

In [73]:
input_df

,brand,product_full_name,description,brand_category,details
0,bottega veneta,Large leather-trimmed linen tote,"beige linen, black leather (calf) open top com...","bags,shoulderbags",This item's measurements are:
68,marc jacobs,The Jewelled Button Sweater,jewel-embellished shoulder buttons add eleganc...,"women,sweaters,cashmere",Roundneck\nShort sleeves\nShoulder buttons\nWo...
150,frame,Les Second - Medium--NOIR,"minimal, modern styling meets refined luxury i...",accessories,NaN
836,sarto by franco sarto,Visa Mule,an asymmetrical topline puts a modern spin on ...,unknown,True to size.
951,banana republic,Madison 12-Hour Loafer Pump,everything you love about our original madison...,unknown,Everything you love about our original Madison...
...,...,...,...,...,...
158987,orseund iris,Night Out open-back ruched satin blouse,cream satin tie and concealed snap fastenings ...,"clothing,tops,blouses","Fits true to size, take your normal size \nDes..."
158989,reformation,Jane Sweater,soft spot.this is a slightly cropped sweater w...,full fashion,NaN
158993,veronica beard,Crosbie High-Rise Wide-Leg,our elongating crosbie jeans feature a flatter...,jeans,NaN
158995,reformation,Jessie High Crop Boot,denim - your uniform since forever.this is a h...,jeans,NaN


In [74]:
styles_pred, occasions_pred = output_df(input_df.fillna(''), input_occasion_df, input_style_df)

In [77]:
styles_out = []
occasions_out = []
for i in range(len(input_df)):
    style_o = []
    occasion_o = []
    for style in styles_pred.keys():
        if styles_pred[style][i]:
            style_o.append(style)
    styles_out.append('/'.join(style_o))
    for occasion in occasions_pred.keys():
        if occasions_pred[occasion][i]:
            occasion_o.append(occasion)
    occasions_out.append('/'.join(occasion_o))

In [78]:
input_df['style'] = styles_out
input_df['occasion'] = occasions_out

## Summary

Our methodology involves preliminary data cleaning (regex, remove stopwords, lemmatization), creating one hot for category and label encoding brands.

We implemented both deep learning & traditional NLP methods with classical ML methods (TF-IDF and word embeddings).

Given the tight time frame, for deep learning, we only concatenated text and used pre-trained embedding.

This way of training also assumed the concatenated input strings are of a single sequence.

Given more time, we can seperate the input strings (description,details,fullname) sequences with categorical variables in our deep learning model architecture.

We used F1-scores as our final evaluation metrics to assess our models because we want to achieve a good balance between precision & recall.